In [ ]:
#takes a df of categories weights, which we will use as a reference  (categories_weights_reference.xlsx )
#the output of this will be one df. For each category:
#for every hour in the period, we will have one row of the weights

 

In [15]:
#load all categories
import pandas as pd
import numpy as np
import string
import random 
import requests
from datetime import datetime
import json
import ast  # Make sure to import the ast module

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
   
 


  
categories_weights_reference =pd.read_excel( 'categories_weights_reference.xlsx' )   
 

print ('done')

  


done


In [16]:
categories_weights_reference

,high_level_category,transaction_categories,years,years_weights,months,months_weights,days,days_weights,hours,hours_weights,day_of_weeks,day_of_weeks_weights
0,Beverage,Coffee,"[2020,2021,2022,2023,2024]","[4,3,2,1,1]","[2,3,4,9,10,11]","[4,3,2,3,4,4]","[12,13,15,20,22,25]","[2,3,2,4,4,4]","[7,8,9]","[5,6,3]","['Tuesday','Wednesday','Thursday']","[3,2,3]"
1,Beverage,Tea,"[2020,2021,2022,2023,2024]","[4,3,2,1,1]","[2,3,4,9,10,11]","[4,3,2,3,4,4]","[2,12,20,23,25,26]","[3,2,2,3,5,2]","[2,5,8,9]","[3,3,3,4]","['Wednesday','Monday','Friday']","[2,4,4]"
2,Beverage,Soft Drinks,"[2020,2021,2022,2023,2024]","[1,2,4,2,3]","[7,8,11,12]","[4,3,4,4]","[1,3,5,16,26,29]","[3,5,4,2,5,5]","[14,15,16]","[3,4,3]","['Friday','Tuesday','Monday']","[4,3,3]"
3,Beverage,Alcohol,"[2020,2021,2022,2023,2024]","[1,2,4,2,3]","[7,8,11,12]","[4,3,4,4]","[24,25,26,27,28,29,30]","[3,4,5,4,3,2,2]","[18,19,20,21]","[3,4,4,3]","['Friday','Saturday']","[4,5]"
4,Beverage,Bottled Water,"[2020,2021,2022,2023,2024]","[3,2,3,3,3]","[1,8,9,10]","[2,4,2,3]","[2,13,15,16,18,19]","[2,5,2,2,4,2]","[2,15,21,22]","[3,2,5,3]","['Sunday','Monday','Saturday']","[4,3,3]"
5,Beverage,Juice,"[2020,2021,2022,2023,2024]","[2,1,3,2,3]","[4,5,7,12]","[2,2,2,2]","[1,3,18,21,22,26]","[4,5,4,2,5,2]","[4,6,7,8]","[5,4,4,3]","['Wednesday','Friday','Thursday']","[3,4,4]"
6,Beverage,Energy Drinks,"[2020,2021,2022,2023,2024]","[1,1,3,4,3]","[1,2,3,4]","[2,3,4,3]","[4,14,15,17,27,29]","[3,3,2,4,3,3]","[0,8,19,21]","[3,5,5,3]","['Wednesday','Saturday','Thursday']","[2,4,2]"
7,Beverage,Wine,"[2020,2021,2022,2023,2024]","[2,1,1,2,1]","[3,4,9,11]","[4,2,4,2]","[24,25,26,27,28,29,30]","[3,4,5,4,3,2,2]","[18,19,20,21]","[3,4,4,3]","['Friday','Saturday']","[4,5]"
8,Beverage,Beer,"[2020,2021,2022,2023,2024]","[1,2,4,2,3]","[7,8,11,12]","[4,3,4,4]","[24,25,26,27,28,29,30]","[3,4,5,4,3,2,2]","[18,19,20,21]","[3,4,4,3]","['Friday','Saturday']","[4,5]"
9,Oil & Gas,Oil,"[2020,2021,2022,2023,2024]","[1,1,3,4,3]","[1,2,3,4]","[2,3,4,3]","[9,12,17,21,23,26]","[2,2,2,2,4,3]","[4,7,14,23]","[5,5,2,5]","['Thursday','Monday','Wednesday']","[3,2,4]"


In [17]:

#the purpoes is to build a function that takes start_date, end_date, excpetions for weights >>> return a complete df with final weights for each hour


def create_hourly_dataframe(start_date, end_date, 
                            years=[], years_weights=[], 
                            months=[], months_weights=[], 
                            days=[], days_weights=[], 
                            hours=[], hours_weights=[], 
                            day_of_weeks=[], day_of_weeks_weights=[], 
                            default_weight=1, 
                           category=''):
    # Generate a date range with hourly frequency
    date_range = pd.date_range(start=start_date, end=end_date, freq='H')

    # Create a DataFrame with the date range
    df = pd.DataFrame(date_range, columns=['datetime'])

    # Add columns for year, month, day_of_week (name), day, and hour
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day_of_week'] = df['datetime'].dt.day_name()
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    df['category']= category

    # Initialize weight columns with default weight
    df['year_weight'] = default_weight
    df['month_weight'] = default_weight
    df['day_weight'] = default_weight
    df['hour_weight'] = default_weight
    df['day_of_week_weight'] = default_weight
     

    # Adjust weights based on specified arguments
    if years and years_weights:
        for year, weight in zip(years, years_weights):
            df.loc[df['year'] == year, 'year_weight'] = weight

    if months and months_weights:
        for month, weight in zip(months, months_weights):
            df.loc[df['month'] == month, 'month_weight'] = weight

    if days and days_weights:
        for day, weight in zip(days, days_weights):
            df.loc[df['day'] == day, 'day_weight'] = weight

    if hours and hours_weights:
        for hour, weight in zip(hours, hours_weights):
            df.loc[df['hour'] == hour, 'hour_weight'] = weight

    if day_of_weeks and day_of_weeks_weights:
        for day_of_week, weight in zip(day_of_weeks, day_of_weeks_weights):
            df.loc[df['day_of_week'] == day_of_week, 'day_of_week_weight'] = weight

    
    df['final_weight'] = (df['year_weight'] * df['month_weight'] * 
                          df['day_weight'] * df['hour_weight'] * 
                          df['day_of_week_weight'])
    return df

 

# start_date = '2024-01-01'
# end_date = '2024-01-2'
 
# df = create_hourly_dataframe(start_date, end_date,
#                              years=[2023], years_weights=[3],
#                              months=[1], months_weights=[2],
#                              days=[1], days_weights=[3],
#                              hours=[0, 1], hours_weights=[2, 3],
#                              day_of_weeks=['Sunday', 'Tuesday'], day_of_weeks_weights=[4,8],
#                              default_weight=1)
# df





In [19]:
#this will loop over all raws in the categories_weights_reference, read weights and then generate on df for all categories
#and their weights per hour

start_date='2020-01-01'
end_date='2025-04-21'
 
result_dfs = []
categories_weights = []

for _, row in categories_weights_reference.iterrows():
   

    years=  ast.literal_eval(row.years) 
    years_weights=ast.literal_eval(row.years_weights) 
    months= ast.literal_eval(row.months) 
    months_weights= ast.literal_eval(row.months_weights) 
    days= ast.literal_eval(row.days) 
    days_weights= ast.literal_eval(row.days_weights) 
    hours= ast.literal_eval(row.hours) 
    hours_weights= ast.literal_eval(row.hours_weights) 
    
    day_of_weeks=  ast.literal_eval(row.day_of_weeks) 
    day_of_weeks_weights=  ast.literal_eval(row.day_of_weeks_weights) 
    category = row.transaction_categories
     
    df = create_hourly_dataframe(
        start_date=start_date,
        end_date=end_date,
        years=years,
         years_weights=years_weights,
        months=months,
        months_weights= months_weights,
        days= days,
        days_weights= days_weights,
        hours= hours,
        hours_weights=hours_weights,
        day_of_weeks=day_of_weeks,
        day_of_weeks_weights=day_of_weeks_weights,
        default_weight=1,
        category =category
    )
 
    result_dfs.append(df)

# Concatenate all the generated DataFrames into one final DataFrame
categories_weights = pd.concat(result_dfs, ignore_index=True)
print('done')

done


In [20]:


print ( len (categories_weights))
 

25382994


In [26]:
# categories_weights.head() [ ['datetime','category','final_weight']]

,datetime,category,final_weight
0,2020-01-01 00:00:00,Coffee,8
1,2020-01-01 01:00:00,Coffee,8
2,2020-01-01 02:00:00,Coffee,8
3,2020-01-01 03:00:00,Coffee,8
4,2020-01-01 04:00:00,Coffee,8


In [25]:
categories_weights.head().columns

Index(['datetime', 'year', 'month', 'day_of_week', 'day', 'hour', 'category',
       'year_weight', 'month_weight', 'day_weight', 'hour_weight',
       'day_of_week_weight', 'final_weight'],
      dtype='object')

In [10]:
#to verify: filter on one category
category='Tikkie'
# datetime= '2024-01-01 01:00:00'
categories_weights.query("category==@category ")

,datetime,year,month,day_of_week,day,hour,category,year_weight,month_weight,day_weight,hour_weight,day_of_week_weight,final_weight
510665,2024-01-01 00:00:00,2024,1,Monday,1,0,Tikkie,1,3,1,1,1,3
510666,2024-01-01 01:00:00,2024,1,Monday,1,1,Tikkie,1,3,1,1,1,3
510667,2024-01-01 02:00:00,2024,1,Monday,1,2,Tikkie,1,3,1,1,1,3
510668,2024-01-01 03:00:00,2024,1,Monday,1,3,Tikkie,1,3,1,5,1,15
510669,2024-01-01 04:00:00,2024,1,Monday,1,4,Tikkie,1,3,1,5,1,15
510670,2024-01-01 05:00:00,2024,1,Monday,1,5,Tikkie,1,3,1,1,1,3
510671,2024-01-01 06:00:00,2024,1,Monday,1,6,Tikkie,1,3,1,1,1,3
510672,2024-01-01 07:00:00,2024,1,Monday,1,7,Tikkie,1,3,1,1,1,3
510673,2024-01-01 08:00:00,2024,1,Monday,1,8,Tikkie,1,3,1,1,1,3
510674,2024-01-01 09:00:00,2024,1,Monday,1,9,Tikkie,1,3,1,1,1,3


In [1]:
#run this to export

cols_to_keep = ['datetime','category','final_weight']
categories_weights_light =  categories_weights.loc[:, cols_to_keep]
 
categories_weights_light.to_csv("categories_weights_3_cols_2020_2025.csv", index=False)
print('done')